# DADOS DO INSTITUTO DE PESQUISA ECONÔMICA APLICADA (IPEA)

Script para extrair dados do Ipeadata por meio de sua API.

In [ ]:
import pandas as pd

Tabela com os dados disponíveis na API

In [ ]:
metadados = pd.read_json("http://www.ipeadata.gov.br/api/odata4/Metadados")

Os dados são disponibilizados em formato JSON; para montar o dataframe, basta transformar a segunda coluna do objeto "metadados" em uma lista de dicionários e depois aplicar "pd.DataFrame()"

In [ ]:
corresp = []
for line in metadados.iloc[:,1]:
    corresp.append(line)
corresp = pd.DataFrame(corresp)

Para ver a lista completa de dados disponíveis na API:
print(corresp["SERNOME"].to_string())

EXEMPLO: EMBI+

In [ ]:
sercodigo_embimais = corresp[corresp["SERNOME"] ==  "EMBI + Risco-Brasil"]["SERCODIGO"].reset_index(drop = True)[0]

api = f"http://www.ipeadata.gov.br/api/odata4/ValoresSerie(SERCODIGO='{sercodigo_embimais}')"
embi_mais = pd.read_json(api)
embi_dict = []
for line in embi_mais["value"]:
    embi_dict.append(line)
embi_mais_df = pd.DataFrame(embi_dict)

# TRANSFORMAR COLUNA DE DATA
embi_mais_df.loc[:,"VALDATA"] = embi_mais_df["VALDATA"].apply(lambda x: pd.to_datetime(x[:10], format = "%Y-%m-%d"))
embi_mais_df.head(5)

# EXTRA: ipeadatapy

Esse pacote, do Luan Borelli (https://www.luanborelli.net/2019/12/31/ipeadatapy/), facilita e extração dos dados do IPEA, pois disponibiliza alguns mecanismos de pesquisa.

In [32]:
import ipeadatapy as idpy

Busca código da série (sem o argumento 'EMBI +', lista todas as séries disponíveis)

In [42]:
idpy.list_series("EMBI +")

,CODE,NAME
7994,JPM366_EMBI366,EMBI + Risco-Brasil


Descreve a série

In [43]:
idpy.describe("JPM366_EMBI366")

,EMBI + Risco-Brasil
Name,EMBI + Risco-Brasil
Code,JPM366_EMBI366
Big Theme,Macroeconômico
Theme,Financeiras
Source,JP Morgan
Source acronym,JP Morgan
Comment,O EMBI+ (Emerging Markets Bond Index Plus) est...
Last update,2021-07-09T16:16:00.45-03:00
Frequency,Diária
Measure,-


Extrai a série

In [44]:
embi_mais_ts = idpy.timeseries("JPM366_EMBI366")
embi_mais_ts

,YEAR,DAY,MONTH,CODE,RAW DATE,VALUE (-)
DATE,,,,,,
1994-04-29,1994,29,4,JPM366_EMBI366,1994-04-29T00:00:00-03:00,1120.0
1994-04-30,1994,30,4,JPM366_EMBI366,1994-04-30T00:00:00-03:00,NaN
1994-05-01,1994,1,5,JPM366_EMBI366,1994-05-01T00:00:00-03:00,NaN
1994-05-02,1994,2,5,JPM366_EMBI366,1994-05-02T00:00:00-03:00,1131.0
1994-05-03,1994,3,5,JPM366_EMBI366,1994-05-03T00:00:00-03:00,1081.0
...,...,...,...,...,...,...
2021-07-02,2021,2,7,JPM366_EMBI366,2021-07-02T00:00:00-03:00,279.0
2021-07-05,2021,5,7,JPM366_EMBI366,2021-07-05T00:00:00-03:00,279.0
2021-07-06,2021,6,7,JPM366_EMBI366,2021-07-06T00:00:00-03:00,287.0
